<a href="https://colab.research.google.com/github/abidrozhan/MidTerm-Deep-Learning/blob/main/AbidRozhan_Midterm_Classificication_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Instalasi dependensi dan pengunduhan dataset
!pip install -q gdown scikit-learn tensorflow

In [2]:
# Cell 2: Memuat dan menyiapkan dataset
import pandas as pd
import numpy as np

# Muat dataset
df = pd.read_csv('clusteringmidterm.csv')

# Hapus kolom CUST_ID jika ada
if 'CUST_ID' in df.columns:
    df.drop(columns=['CUST_ID'], inplace=True)

# Cek kolom BALANCE
assert 'BALANCE' in df.columns, 'Kolom BALANCE tidak ditemukan'

# Membuat target label HighBalance
df['HighBalance'] = (df['BALANCE'] >= df['BALANCE'].median()).astype(int)

# Pisahkan fitur dan target
X = df.drop(columns=['HighBalance'])
y = df['HighBalance']

print('Dataset shape:', df.shape)
print('Distribusi label:')
print(y.value_counts())

Dataset shape: (8950, 18)
Distribusi label:
HighBalance
0    4475
1    4475
Name: count, dtype: int64


In [3]:
# Cell 3: Pra-pemrosesan dan pembagian data
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# Semua fitur numerik
numeric_cols = X.columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), numeric_cols)
    ],
    remainder='drop'
)

# Imputasi
X_imputed = preprocessor.fit_transform(X)

# Standarisasi
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y.values, test_size=0.2, random_state=42, stratify=y
)

print('Train shape:', X_train.shape, y_train.shape)
print('Test shape:', X_test.shape, y_test.shape)

Train shape: (7160, 17) (7160,)
Test shape: (1790, 17) (1790,)


In [4]:
# Cell 4: Membangun dan melatih model jaringan saraf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Arsitektur model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Kompilasi model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Pelatihan
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=2
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
90/90 - 4s - 42ms/step - accuracy: 0.8488 - loss: 0.3638 - val_accuracy: 0.9316 - val_loss: 0.1824
Epoch 2/50
90/90 - 2s - 17ms/step - accuracy: 0.9272 - loss: 0.1821 - val_accuracy: 0.9511 - val_loss: 0.1270
Epoch 3/50
90/90 - 1s - 7ms/step - accuracy: 0.9436 - loss: 0.1400 - val_accuracy: 0.9714 - val_loss: 0.0872
Epoch 4/50
90/90 - 0s - 4ms/step - accuracy: 0.9569 - loss: 0.1074 - val_accuracy: 0.9791 - val_loss: 0.0660
Epoch 5/50
90/90 - 0s - 3ms/step - accuracy: 0.9609 - loss: 0.0965 - val_accuracy: 0.9784 - val_loss: 0.0620
Epoch 6/50
90/90 - 0s - 4ms/step - accuracy: 0.9684 - loss: 0.0789 - val_accuracy: 0.9749 - val_loss: 0.0559
Epoch 7/50
90/90 - 0s - 4ms/step - accuracy: 0.9672 - loss: 0.0781 - val_accuracy: 0.9825 - val_loss: 0.0442
Epoch 8/50
90/90 - 0s - 4ms/step - accuracy: 0.9743 - loss: 0.0622 - val_accuracy: 0.9860 - val_loss: 0.0368
Epoch 9/50
90/90 - 1s - 6ms/step - accuracy: 0.9761 - loss: 0.0606 - val_accuracy: 0.9832 - val_loss: 0.0404
Epoch 10/50
90/90

In [5]:
# Cell 5: Evaluasi model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Prediksi probabilitas dan kelas
y_pred_prob = model.predict(X_test).flatten()
y_pred = (y_pred_prob >= 0.5).astype(int)

# Hitung metrik
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_prob)

print('Accuracy:', acc)
print('Precision:', prec)
print('Recall   :', rec)
print('F1 Score:', f1)
print('ROC-AUC :', auc)

# Rincian laporan klasifikasi
from sklearn.metrics import classification_report
print('Classification Report:')
print(classification_report(y_test, y_pred))

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.9910614525139665
Precision: 0.9899665551839465
Recall   : 0.9921787709497206
F1 Score: 0.9910714285714286
ROC-AUC : 0.9997540651040855
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       895
           1       0.99      0.99      0.99       895

    accuracy                           0.99      1790
   macro avg       0.99      0.99      0.99      1790
weighted avg       0.99      0.99      0.99      1790

